<a href="https://colab.research.google.com/github/charlberg/Constraint-Modelling-Models/blob/main/Scheduling%20Challenge/Students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet pandas openpyxl

Now we can read in data from an Excel file, using pandas.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas
data = pandas.read_excel('/content/drive/MyDrive/Colab Notebooks/Discrete Optimization/Students.xlsx', sheet_name=None)

The file has two worksheets in it, one with student information.

In [ ]:
students = data['Students']
students

,Student ID,First Name,Last Name,GPA,Major
0,1,Emma,Johnson,3.8,CS
1,2,Liam,Smith,3.5,CS
2,3,Olivia,Williams,3.9,CS
3,4,Noah,Jones,3.7,Maths
4,5,Ava,Brown,3.2,CS
5,6,Isabella,Davis,3.6,CS
6,7,Sophia,Miller,2.9,CS
7,8,Mia,Wilson,3.3,Maths
8,9,Harper,Moore,3.1,CS
9,10,Benjamin,Taylor,3.8,Maths


And another one that has the "forbidden pairs" in it.

In [ ]:
forbiddens = data['Forbidden pairs']
forbiddens

,Student 1,Student 2
0,1,3
1,1,4
2,7,13
3,8,17
4,3,21


Let's preprocess the data and prepare it for the model.

In [ ]:
import math

names = {}
gpas = {}
majors = {}
pots = {}

# for line in students.values.tolist():
#     [id, name, surname, gpa, major] = line
#     gpas[id] = int(gpa * 10)
#     majors[id] = 0 if major == "CS" else 1

studentsSortedByGPA = sorted(students.values.tolist(),
                                key = lambda line: line[3])

for index, line in enumerate(studentsSortedByGPA):
    [id, name, surname, gpa, major] = line
    gpas[id] = int(gpa * 10)
    majors[id] = 0 if major == "CS" else 1
    pots[id] = 1 + math.floor(index / 4)
    names[id] = f'[{id}] {name} {surname} ({major} - {gpa} - pot {pots[id]})'

nbStudents = len(students)


Installing and loading the Conjure notebook extension

In [ ]:
%pip install --quiet git+https://github.com/conjure-cp/conjure-notebook.git@v0.0.10
%load_ext conjure

  Preparing metadata (setup.py) ... done
The conjure extension is already loaded. To reload it, use:
  %reload_ext conjure


In [ ]:
%reload_ext conjure

<IPython.core.display.Javascript object>

Error while initializing extension: cannot run conjure.


This extension contains a few _magic_ commands, for example `%%conjure`. Also see `%conjure_help`.


These are the parameters of our model.

In [ ]:
%%conjure

given nbStudents : int
given gpas : function (total) int(1..nbStudents) --> int
given majors : function (total) int(1..nbStudents) --> int(0,1)
given pots : function (total) int(1..nbStudents) --> int(1..6)


UsageError: Cell magic `%%conjure` not found.


And we are trying to find a partitioning of the students.

A partition is a _mutually exclusive_ and _collectively exhaustive_ collection of parts.

Here, we say each part will contain 6 items it it. We know there are 6 pots in this problem. We could have easily parameterised the model over the number of pots, too.

This implies that there will be 4 parts. We can provide this information as well, using `numParts 4` as a domain attribute. We will leave this out for now.

In [ ]:
%%conjure+

find groups : partition (partSize 6) from int(1..nbStudents)


We have a solution now!

It is a solution to an incomplete model however. It doesn't enfore any of our constraints yet. Let's format it sligtly better using Python.

In [ ]:
for n, group in enumerate(groups):
    print(f'# Group {n+1}')
    for s in group:
        print(names[s])
    print()

This is a useful bit of functionality, let's define a Python function that can print the solution this way.

In [ ]:
def printSolution():
    # this is not best practice in general Python programming
    # groups is a global variable
    # however, it's probably fine in a notebook environment
    for n, group in enumerate(groups):
        print(f'# Group {n+1}')
        for s in group:
            print(names[s])
        print()

OK, let's focus on the intermediate solution again. In group 1, we have two people from pot 4. This is not good.

Let's enforce the condition that there has to be a member from each pot in every part of the partition.

In [ ]:
%%conjure+

$ one member from each pot
such that
    forAll group in groups .
        forAll pot : int(1..6) .
            exists s in group .
                pots(s) = pot


In [ ]:
printSolution()

Better.

Now each group has exactly one member from each pot.

But group 1 has a single Maths major in it. Noah might feel lonely, let's enforce the condition that there cannot be a single person from any particular major.

This means a group can contain all CS or all Maths students, but if there is a mixture there will be at least 2 students from each major.

In [ ]:
%%conjure+

$ no single major
such that
    forAll group in groups .
        forAll major : int(0,1) .
            sum([ toInt(majors(s) = major) | s <- group ]) != 1

In [ ]:
printSolution()

We have a list of people who cannot be in the same group for some reason. Let's remember who these people were.

In [ ]:
forbiddens = forbiddens.values.tolist()
forbiddens

Not good. 1 and 3 (Emma and Olivia) are not supposed to be in the same group, but they are.

Let's add another constraint. Pairs of people listed together in the forbidden list must be in different parts of the partition.

You can learn more about the apart constraint expression later. For now, the short version is that it takes two arguments, a set of values and a partition. Items in the set of values must not be together in the same part. Exactly what we need.

In [ ]:
%%conjure+

given forbiddens : set of (int, int)
such that
    forAll (a,b) in forbiddens .
        apart({a,b}, groups)

In [ ]:
printSolution()

Looks good!

This is not a unique solution.

More constraints can be added.

An objective can be added (for example one to do with average gpa of a group?).

Or more solutions can be enumerated.

Stay tuned...